In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%matplotlib inline

In [52]:
import nltk
import re
import os
import pandas as pd
import numpy as np
import scipy as sp
import string
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
# from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score
import datetime

# Read Data

In [5]:
df = pd.read_csv('train_2.csv')

df_test = pd.read_csv("test.csv")
df_test = df_test['Page content']
print(df['Page content'].iloc[2])

<html><head><div class="article-info"><span class="byline "><a href="/author/sam-laird/"><img alt="2016%2f09%2f15%2f63%2fhttpsd2mhye01h4nj2n.cloudfront.netmediazgkymde1lza2.9814b" class="author_image" src="http://i.amz.mshcdn.com/-qaMPB8aiQeIaoBhqlU0OLjA07A=/90x90/2016%2F09%2F15%2F63%2Fhttpsd2mhye01h4nj2n.cloudfront.netmediaZgkyMDE1LzA2.9814b.jpg"/></a><span class="author_name">By <a href="/author/sam-laird/">Sam Laird</a></span><time datetime="Wed, 07 May 2014 19:15:20 +0000">2014-05-07 19:15:20 UTC</time></span></div></head><body><h1 class="title">Ballin': 2014 NFL Draft Picks Get to Choose Their Own Walk-Out Music</h1><figure class="article-image"><img alt="This Dec. 8, 2012 file photo shows Texas A&amp;M quarterback Johnny Manziel gesturing as he talks to reporters after winning the Heisman Trophy in New York." class="microcontent" data-fragment="lead-image" data-image="http://i.amz.mshcdn.com/CADSAjPf_UHvl5Z9OqlkPWABIGU=/950x534/2014%2F05%2F07%2Fff%2FJohnnyOK.cae2a.jpg" data-micro

# Define Preprocessor and Vectorizer

In [6]:
#preprocessor for all text
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    
    return text
# print(preprocessor(df.loc[0,'Page content']))

In [7]:
def preprocessor_title(X):
    ret=[]
    for text in X:
        titles = BeautifulSoup(text, 'html.parser')
        titles = titles.find_all('h1', class_='title')
        t=""
        for title in titles:
            t+=str(title.getText())
        r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
        emoticons = re.findall(r, t)
        t = re.sub(r, '', t)
        t = re.sub('[\W]+', ' ', t.lower()) + ' ' + ' '.join(emoticons).replace('-','')
        ret.append(t)
    return ret

In [8]:
def preprocessor_topic(X):
    ret=[]
    for text in X:
        topics = BeautifulSoup(text, 'html.parser')
        topics = topics.find_all(class_='article-topics')
        t=""
        for topic in topics:
            t+=str(topic.getText())

        r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
        emoticons = re.findall(r, t)
        t = re.sub(r, '', t)
        t = re.sub('[\W]+', ' ', t.lower()) + ' ' + ' '.join(emoticons).replace('-','')
        t = t[7:] # remove " Topics"
        ret.append(t)
    
    return ret

In [9]:
def preprocessor_channel(X):
    ret=[]
    for text in X:
        channel_flag = 0
        channels = BeautifulSoup(text, 'html.parser')
        channels = channels.find_all('article')
        for channel in channels:
            if 'data-channel' in channel.attrs:
                channel_flag = 1
                ret.append(channel['data-channel'])
        if channel_flag == 0:
            ret.append(" entertainment nfl sports  ")
    return ret


In [10]:
def preprocessor_weekday(X):
    weekday = []
    for i in range(len(X)):
        if 'datetime="' in X[i]:
            split_data = X[i].split('datetime="')[1].split(",")[0]
            weekday.append(split_data)
        else:
            weekday.append("Mon")
    return weekday

In [11]:
def preprocessor_time(X):
    ret=np.array([0, 0, 0, 0, 0, 0])
    z=0
    for text in X:
        one_doc = []
        times = BeautifulSoup(text, 'html.parser')
        times = times.find_all('time')
        
        
        sp = times[0].getText().split(' ')
        if(len(sp)!=1):
            date = sp[0].split('-')
            moment = sp[1].split(':')

            date = [int(d) for d in date]
            moment = [int(m) for m in moment]

            one_doc = np.hstack((np.array(date), np.array(moment)))
            ret = np.vstack((ret, one_doc))
        else:
            temp=np.array([2010, 9, 29, 8, 0, 0])
            ret = np.vstack((ret, temp))
    ret = np.delete(ret, 0, axis = 0)
    return ret

In [12]:
def preprocessor_textlen(X):
    L=[]
    for text in X:
        inners = BeautifulSoup(text, 'html.parser')
        inners = inners.find_all('section', class_='article-content')
        #print(inners)
        L.append(len(inners[0].getText()))
    L = [int(l) for l in L]
    leng = len(L)
    L = np.array(L).reshape(leng, 1)
    return L


In [13]:
def preprocessor_src(X):
    ret=[]
    for text in X:
        image_count = (text.count("src"))
        ret.append(image_count)
    leng = len(ret)
    ret = np.array(ret).reshape(leng, 1)
    return ret

In [14]:
stop = stopwords.words('english')
def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

## MERGE FEATURE

In [15]:
TITLE = preprocessor_title(df['Page content'])    # (27643,  ) str, each is a str of all vacabulary in title of a document
TOPIC = preprocessor_topic(df['Page content'])    # (27643,  ) str, each is a str of all vacabulary in topic of a document
CHANNEL = preprocessor_channel(df['Page content'])# (27643,  ) str, each is one vacabulary of channel of a document
WEEK = preprocessor_weekday(df['Page content'])   # (27643,  ) str, each is one vacabulary from Mon to Sat
TIME = preprocessor_time(df['Page content'])      # (27643, 6) int, for each doc, (year, month, date, hour, minute, sec) 
TEXTLEN = preprocessor_textlen(df['Page content'])# (27643,  ) int, article-content length of each document
SRC = preprocessor_src(df['Page content'])        # (27643,  ) int, #images of each document 

In [16]:
WEEK_dum = pd.get_dummies(WEEK)
WEEK_dum = np.array(WEEK_dum)
# one hot of (27643, 7)
CHANNEL[0]='travel-leisure'
CHANNEL_dum = pd.get_dummies(CHANNEL)
CHANNEL_dum = np.array(CHANNEL_dum)
# one hot of (27643, 33)

In [17]:
TITLE_test = preprocessor_title(df_test)    # (27643,  ) str, each is a str of all vacabulary in title of a document
TOPIC_test = preprocessor_topic(df_test)    # (27643,  ) str, each is a str of all vacabulary in topic of a document
CHANNEL_test = preprocessor_channel(df_test)# (27643,  ) str, each is one vacabulary of channel of a document
WEEK_test = preprocessor_weekday(df_test)   # (27643,  ) str, each is one vacabulary from Mon to Sat
TIME_test = preprocessor_time(df_test)      # (27643, 6) int, for each doc, (year, month, date, hour, minute, sec) 
TEXTLEN_test = preprocessor_textlen(df_test)# (27643,  ) int, article-content length of each document
SRC_test = preprocessor_src(df_test)

In [18]:
WEEK_dum_test = pd.get_dummies(WEEK_test)
WEEK_dum_test = np.array(WEEK_dum_test)

CHANNEL_test[0]='bus'
CHANNEL_test[1]='comics'
CHANNEL_test[2]='howto'
CHANNEL_test[4]='viral'
CHANNEL_dum_test = pd.get_dummies(CHANNEL_test)
CHANNEL_dum_test = np.array(CHANNEL_dum_test)

In [19]:
X_train = []
for i in range(len(TITLE)):
    temp=TITLE[i]+" "+TOPIC[i]
    X_train.append(temp)
    
X_test = []
for i in range(len(TITLE_test)):
    temp=TITLE_test[i]+" "+TOPIC_test[i]
    X_test.append(temp)


In [36]:
merge_data=np.hstack((np.array(X_train), np.array(X_test)))

vect = CountVectorizer(tokenizer=tokenizer_stem_nostop,
                       max_features=2000,
                       lowercase=False
                      ).fit(merge_data)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)
#print(X_train_vectorized.shape)
# concatenate CHANNEL_dum, WEEK_dum, TEXTLEN, TIME, and X_train_vectorized

Y_train=df['Popularity']

In [37]:
dum_temp1 = np.hstack((WEEK_dum, CHANNEL_dum))
dum_temp2 = np.hstack((TEXTLEN, TIME))
dum = np.hstack((dum_temp1, dum_temp2))
dum = np.hstack((dum, SRC))
X_train_vectorized = np.hstack((X_train_vectorized.toarray(), dum))


dum_temp1 = np.hstack((WEEK_dum_test, CHANNEL_dum_test))
dum_temp2 = np.hstack((TEXTLEN_test, TIME_test))
dum = np.hstack((dum_temp1, dum_temp2))
dum = np.hstack((dum, SRC_test))
X_test_vectorized = np.hstack((X_test_vectorized.toarray(), dum))

In [38]:
print(X_train_vectorized.shape)
print(X_test_vectorized.shape)

(27643, 2049)
(11847, 2049)


# Start Training

In [42]:
best_val_score=0
for i in range(100):
    clf = LGBMClassifier(max_depth=9, n_estimators=50)
    x_train, X_val, y_train, y_val = train_test_split(X_train_vectorized, Y_train, test_size=0.2, shuffle=True)
    clf.fit(x_train, y_train)
    print("round:", i+1)
    train_score=clf.score(x_train, y_train)
    val_score=clf.score(X_val, y_val)
#     train_score=roc_auc_score(y_train, clf.predict_proba(x_train)[:,1])
    
    print("train: ", train_score)
    print("valid: ", val_score)
    print("auc: ", roc_auc_score(y_val, clf.predict_proba(X_val)[:,1]))
    
    if(val_score>best_val_score):
        pred=clf.predict_proba(X_test_vectorized)[:,1]
        index=[]
        for i in range(len(pred)):
            index.append(27643+i)
        dict = {'Id': index, 'Popularity': pred}
        predict = pd.DataFrame(dict) 
        predict.to_csv('prediction.csv', index=False)
        best_val_score=val_score
        print("===============save model===============")

round: 1
train:  0.6040969521570047
valid:  0.5581479471875566
auc:  0.5954296780440571
===============save model===============
round: 2
train:  0.6041421723794881
valid:  0.545849158979924
auc:  0.5693716708764675
round: 3
train:  0.6155376684453288
valid:  0.5574244890576958
auc:  0.5675336221321066
round: 4
train:  0.6109252057520123
valid:  0.5735214324470971
auc:  0.5832220114895138
===============save model===============
round: 5
train:  0.6078502306231347
valid:  0.5550732501356483
auc:  0.5793123916173815
round: 6
train:  0.6098399204124084
valid:  0.571351058057515
auc:  0.5919691862911357
round: 7
train:  0.6106538844171113
valid:  0.5673720383432809
auc:  0.5768908393419708
round: 8
train:  0.6098399204124084
valid:  0.5614035087719298
auc:  0.5820098715174056
round: 9
train:  0.6066745048385638
valid:  0.5677337674082112
auc:  0.5934711802570334
round: 10
train:  0.6031473274848512
valid:  0.5465726171097848
auc:  0.5736021798593309
round: 11
train:  0.6050013566066745
va